In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project-course1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: Datascience
Azure region: westus2
Subscription id: 28f596d1-e824-4442-a8d2-cfc2169974a2
Resource group: AzureML-Elijah


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Create compute cluster
from azureml.core.compute import AmlCompute, ComputeTarget

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpu-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout.
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target. just use it. cpu-cluster


In [4]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pip:
  - azureml-defaults
  - packaging

Overwriting conda_dependencies.yml


In [5]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

In [35]:
# Create a folder
from azureml.core import ScriptRunConfig
project_folder = './training'
os.makedirs(project_folder, exist_ok=True)

import shutil

shutil.copy('train.py', project_folder)

'./training/train.py'

In [46]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "--penalty": choice('l1', 'l2'),
        "--C": uniform(0.05, 1.0),
        "--solver": choice('lbfgs', 'liblinear'),
        "--max_iter": choice(100, 200, 300, 400, 500, 1000)
    }
)

# Specify a Policy | See docs here: https://bit.ly/3hfPGRM
early_termination_policy = BanditPolicy(slack_factor = 0.2, evaluation_interval = 10, delay_evaluation = 20)

src = ScriptRunConfig(source_directory=project_folder,
                      script='train.py',
                      arguments=['--C', 1.0, '--max_iter', 100, '--penalty', 'l2', '--solver', 'lbfgs'],
                      compute_target=compute_target,
                      environment=sklearn_env)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                        policy=early_termination_policy,
                                        hyperparameter_sampling=ps, 
                                        primary_metric_name="AUC_weighted",
                                        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                        max_total_runs=12,
                                        max_concurrent_runs=4)

In [47]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
experiment = exp
hyperdrive_run = experiment.submit(hyperdrive_config)




In [48]:
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO',…

In [49]:
hyperdrive_run.wait_for_completion(show_output=True)


RunId: HD_ebf12f8b-ac0e-4603-b48a-6efbe1a6dc4d
Web View: https://ml.azure.com/experiments/udacity-project-course1/runs/HD_ebf12f8b-ac0e-4603-b48a-6efbe1a6dc4d?wsid=/subscriptions/28f596d1-e824-4442-a8d2-cfc2169974a2/resourcegroups/AzureML-Elijah/workspaces/Datascience

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-24T05:48:16.583312][API][INFO]Experiment created<END>\n""<START>[2020-12-24T05:48:17.128463][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-12-24T05:48:17.374562][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-12-24T05:48:18.4991387Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_ebf12f8b-ac0e-4603-b48a-6efbe1a6dc4d
Web View: https://ml.azure.com/experiments/udacity-project-course1/runs/HD_ebf12f8b-ac0e-4603-b48a-6efbe1a6dc4d?wsid=/subscriptions/28f596d1

{'runId': 'HD_ebf12f8b-ac0e-4603-b48a-6efbe1a6dc4d',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-12-24T05:48:16.25764Z',
 'endTimeUtc': '2020-12-24T06:00:00.725478Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'properties': {'primary_metric_config': '{"name": "AUC_weighted", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'eb0737df-9a15-4934-a2ea-543d16337aba',
  'score': '0.8046988674175245',
  'best_child_run_id': 'HD_ebf12f8b-ac0e-4603-b48a-6efbe1a6dc4d_8',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://datascience6246379368.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_ebf12f8b-ac0e-4603-b48a

In [50]:
assert(hyperdrive_run.get_status() == "Completed")

In [52]:
import joblib
# Get your best run and save the model from that run.

## Save best model
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

print(best_run.get_file_names())
model = best_run.register_model(model_name='udacity-logreg', model_path='outputs/model.joblib')

['--C', '1', '--max_iter', '100', '--penalty', 'l2', '--solver', 'lbfgs', '--C', '0.7782702392085862', '--max_iter', '500', '--penalty', 'l2', '--solver', 'lbfgs']
['azureml-logs/55_azureml-execution-tvmps_d367e49e06c9debd354ef59fbbc8edebd06a3edc75589d0b604a06cdf054d6a7_d.txt', 'azureml-logs/65_job_prep-tvmps_d367e49e06c9debd354ef59fbbc8edebd06a3edc75589d0b604a06cdf054d6a7_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_d367e49e06c9debd354ef59fbbc8edebd06a3edc75589d0b604a06cdf054d6a7_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/100_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_7b86b04f-37e9-4e10-8263-c2cffdc927ad.jsonl', 'logs/azureml/dataprep/python_span_l_7b86b04f-37e9-4e10-8263-c2cffdc927ad.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


## AUTOML run

In [3]:
# see here for AutoML configs = https://bit.ly/2WJZOZr
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

from azureml.core import Dataset
dataUrl = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'

ds = Dataset.Tabular.from_delimited_files(path = dataUrl)

In [4]:
from train import clean_data
import pandas as pd
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

training_data = pd.concat([x_train, y_train], axis=1, join='inner')
validation_data = pd.concat([x_test, y_test], axis=1, join='inner')
label = 'y'


In [5]:
# register the pandas dataframe as AzureML dataset | See how to register this in workspace for other users and other experiments: https://bit.ly/3aMpTPI
data_folder = './data'
os.makedirs(data_folder, exist_ok=True)

local_training_path = 'data/training_data.csv'
local_validation_path = 'data/validation_data.csv'
training_data.to_csv(local_training_path)
validation_data.to_csv(local_validation_path)

In [6]:
# get the datastore to upload prepared data
datastore = ws.get_default_datastore()


In [7]:
# upload the local files from src_dir to the target_path in datastore
datastore.upload(src_dir='data', target_path='data')

# create a dataset referencing the cloud location
training_data = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/training_data.csv'))])
validation_data = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/validation_data.csv'))])

Uploading an estimated of 2 files
Target already exists. Skipping upload for data/training_data.csv
Target already exists. Skipping upload for data/validation_data.csv
Uploaded 0 files


In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    debug_log = 'automl_errors.log',
    primary_metric='AUC_weighted',
    # experiment_exit_score = 0.99,
    training_data=training_data,
    label_column_name=label,
    validation_data = validation_data,
    # n_cross_validations=10,
    enable_voting_ensemble=True,
    enable_stack_ensemble=True,
    compute_target=compute_target,
    # enable_onnx_compatible_models=True
)

In [9]:
# Submit your automl run
# First - choose a name for the experiment.

experiment = exp

automl_run = experiment.submit(automl_config, show_output = True)
automl_run

Running on remote.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project-course1,AutoML_7845fee6-2af6-4c37-8812-4199353cabc7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [13]:
# Widget for Monitoring Runs

from azureml.widgets import RunDetails
RunDetails(automl_run).show() 

automl_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 'sd…

In [15]:
# Retrieve and save your best automl model.
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)

model = best_run.register_model(model_name='udacity-automl.pkl', model_path='./outputs')

Run(Experiment: udacity-project-course1,
Id: AutoML_7845fee6-2af6-4c37-8812-4199353cabc7_32,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               objective='reg:logistic',
                                                                                               random_state=0,
                       

In [16]:
# See best run based on accuracy
lookup_metric = "accuracy"
best_run, fitted_model = automl_run.get_output(metric = lookup_metric)
print(best_run)
print(fitted_model)

Run(Experiment: udacity-project-course1,
Id: AutoML_7845fee6-2af6-4c37-8812-4199353cabc7_30,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('StandardScalerWrapper',
                 <azureml...
                                   colsample_bytree=0.8, eta=0.3, gamma=0.1,
                                   learning_rate=0.1, max_delta_step=0,
                                   max_depth=6, max_leaves=0,
                          

In [18]:
# Delete compute target after use
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

